# Candlestick Chart of Stock Data

In [1]:
import pandas as pd
import numpy as np
import requests
import bokeh.plotting as bk
from bokeh.models import Label, HoverTool, BoxZoomTool, PanTool, ZoomInTool, ZoomOutTool, ResetTool
API_URL = 'https://api.iextrading.com/1.0'

In [2]:
res = requests.get(f'{ API_URL }/stock/MSFT/chart/5y')

In [3]:
data = res.json()

In [4]:
df = pd.DataFrame(data)

In [5]:
df.sample(2)

,change,changeOverTime,changePercent,close,date,high,label,low,open,unadjustedVolume,volume,vwap
271,0.136656,0.314767,0.324,42.2628,2015-01-20,42.4997,"Jan 20, 15",41.5157,42.1808,36161860,36161860,42.0713
258,-0.391743,0.332621,-0.906,42.8367,2014-12-30,43.3834,"Dec 30, 14",42.6728,43.2194,16384692,16384692,42.9634


In [6]:
seqs = np.arange(df.shape[0])
df['seqs'] = pd.Series(seqs)

In [7]:
df.sample(2)

,change,changeOverTime,changePercent,close,date,high,label,low,open,unadjustedVolume,volume,vwap,seqs
1149,-0.515778,2.237184,-0.493,104.0583,2018-07-16,104.9609,"Jul 16, 18",103.6665,104.5443,21786912,21786912,104.2402,1149
474,0.501966,0.588190,0.993,51.0519,2015-11-06,51.1077,"Nov 6, 15",50.1632,50.2757,32851204,32851204,50.7453,474


In [8]:
df['changePercent'] = df['changePercent'].apply(lambda x: str(x) + '%')

In [9]:
df['mid'] = df.apply(lambda x: (x['open'] + x['close']) / 2, axis=1)

In [10]:
df['height'] = df.apply(
    lambda x: x['close'] - x['open'] if x['close'] != x['open'] else 0.01,
    axis=1
)

In [11]:
inc = df.close > df.open
dec = df.close < df.open
w = .3

In [12]:
sourceInc = bk.ColumnDataSource(df.loc[inc])
sourceDec = bk.ColumnDataSource(df.loc[dec])

In [13]:
hover = HoverTool(
    tooltips=[
        ('Date', '@date'),
        ('Low', '@low'),
        ('High', '@high'),
        ('Open', '@open'),
        ('Close', '@close'),
        ('Percent', '@changePercent'),
    ]
)

In [14]:
TOOLS = [hover, BoxZoomTool(), PanTool(), ZoomInTool(), ZoomOutTool(), ResetTool()]

In [15]:
p = bk.figure(plot_width=1200, plot_height=800, title='Microsoft', tools=TOOLS, toolbar_location='above')

In [16]:
p.xaxis.major_label_orientation = np.pi/4
p.grid.grid_line_alpha = w

In [17]:
descriptor = Label(x=70, y=70, text='Your label text goes here')
p.add_layout(descriptor)

In [18]:
p.segment(df.seqs[inc], df.high[inc], df.seqs[inc], df.low[inc], color='green')

GlyphRenderer(id='1043', ...)

In [19]:
p.segment(df.seqs[dec], df.high[dec], df.seqs[dec], df.low[dec], color='red')

GlyphRenderer(id='1050', ...)

In [20]:
p.rect(x='seqs', y='mid', width=w, height='height', fill_color='red', line_color='red', source=sourceDec)

GlyphRenderer(id='1057', ...)

In [21]:
p.rect(x='seqs', y='mid', width=w, height='height', fill_color='green', line_color='green', source=sourceInc)

GlyphRenderer(id='1064', ...)

In [22]:
# bk.show(p)

In [25]:
bk.save(p, './candlestick.html', title='5yr viz')

/Users/sjschmidt/.local/share/virtualenvs/data_analysis-GQRDbUGF/lib/python3.6/site-packages/bokeh/io/saving.py:127: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")


'/Users/sjschmidt/CodeFellows/401/python/lectures/data_analysis/candlestick.html'